**EDA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", color_codes=True)
sns.set(font_scale=1)


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools


from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_json("../input/train.json")
train.head()

In [ ]:
test = pd.read_json("../input/test.json")
test.head()


Data Types
----------------



In [ ]:
train.dtypes


In [ ]:
test.shape

Description
---------------------------

In [ ]:
##Describe gives statistical information about numerical columns in the dataset
train.describe()

Wordclouds for description,addresses
---------------------------

In [ ]:
#Preprocessing
from wordcloud import WordCloud
text = ''
text_dispadd = ''
text_street = ''
text_desc =''
for ind, row in train.iterrows():
    for feature in row['features']:
        text = " ".join([text, "_".join(feature.strip().split(" "))])
    text_dispadd = " ".join([text_dispadd,"_".join(row['display_address'].strip().split(" "))])
    text_street = " ".join([text_street, row['street_address']])
    text_desc=" ".join([text_desc, row['description']])
text = text.strip()
text_dispadd = text_dispadd.strip()
text_street = text_street.strip()
text_desc = text_desc.strip()
text_dispadd

**Display Address**
-------------------

In [ ]:
import random



# wordcloud for display address
plt.figure(figsize=(12,6))
wc = WordCloud(background_color='white', width=600, height=300,
                      max_font_size=50, max_words=40).generate(text_dispadd)
plt.imshow(wc.recolor( random_state=3))
#plt.imshow(wc)
plt.title("Display Address", fontsize=20)
plt.axis("off")
plt.show()

**Features**
------------

In [ ]:
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300,
                      max_font_size=50, max_words=40).generate(text)
plt.imshow(wc.recolor( random_state=3))
plt.imshow(wordcloud)
plt.title("Features", fontsize=20)
plt.axis("off")
plt.show()

**Street Address**
------------------

In [ ]:
#street address
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300,
                      max_font_size=50, max_words=40).generate(text_street)
wordcloud.recolor( random_state=3)
plt.imshow(wordcloud)
plt.title("Street Address", fontsize=20)
plt.axis("off")
plt.show()

**Description**
===============

In [ ]:
plt.figure(figsize=(12,6))
wordcloud = WordCloud(background_color='white', width=600, height=300,
                      max_font_size=50, max_words=40).generate(text_desc)
wordcloud.recolor(random_state=3)
plt.imshow(wordcloud)
plt.title("Description", fontsize=20)
plt.axis("off")
plt.show()

In [ ]:
int_plot = sns.countplot(train["interest_level"])

Very few listings are with high interest.

*Price*
-------

In [ ]:
price_hist = sns.distplot(train["price"])

There seem to be outliers in the data

In [ ]:
ulimit = np.percentile(train.price.values, 99)
train.ix[train['price']>ulimit,'price'] = ulimit


plt.figure(figsize=(6, 8))
plt.scatter(range(train.shape[0]), train["price"].values,color='red')
plt.ylabel("Price")
plt.title("Distribution of Price");

In [ ]:
price_hist = sns.distplot(train["price"])

In [ ]:
order = ['low', 'medium', 'high']
sns.stripplot(train["interest_level"],train["price"],order=order,jitter=True)
plt.title("Price Vs Interest_level");

BedRooms
--------------------

In [ ]:
train["bedrooms"].value_counts().plot(kind='bar')
plt.ylabel("Frequency")
plt.xlabel("Number of Bedrooms");

In [ ]:
sns.stripplot(y="bedrooms", x="interest_level",data=train,jitter=True,order=order);

*Number of BathRooms*   bathrooms and bedrooms are cotrelated with price
---------------------

In [ ]:
sns.stripplot(x="bathrooms", y="price"
              ,data=train,jitter=True);

In [ ]:
sns.stripplot(y="bathrooms", x="interest_level",data=train,jitter=True,order=order);

Removing outliers

In [ ]:
ulimit = np.percentile(train.bathrooms.values, 99)
train.ix[train['bathrooms']>ulimit,'bathrooms'] = ulimit

In [ ]:

sns.stripplot(y="bathrooms", x="interest_level",data=train,jitter=True,order=order);

**Listing Id**
==============

In [ ]:
sns.stripplot(train["interest_level"],train["listing_id"],order=order)
plt.title("L-id Vs Interest_level");

Listing ids above 740000 seem to have low interest level


  

**Feature engineering**
=======================

**Aggregate on Text data**
--------------------------

In [ ]:
train['num_photos'] = train['photos'].apply(len)
train['num_features'] = train['features'].apply(len)
train['num_description_words'] = train['description'].apply(lambda x: len(x.split(' ')))
train['num_description_len'] = train['description'].apply(len)

**Breaking created date column**
--------------------------------

In [ ]:
train['created'] = pd.to_datetime(train['created'])
train['date'] = train['created'].dt.date
train["year"] = train["created"].dt.year
train['month'] = train['created'].dt.month
train['day'] = train['created'].dt.day
train['hour'] = train['created'].dt.hour
train['weekday'] = train['created'].dt.weekday
train['week'] = train['created'].dt.week
train['quarter'] = train['created'].dt.quarter
train['weekend'] = ((train['weekday'] == 5) & (train['weekday'] == 6))
train['wd'] = ((train['weekday'] != 5) & (train['weekday'] != 6))

**Dates**
-------

In [ ]:
cnt_srs = train['date'].value_counts()

plt.figure(figsize=(12,4))
ax = plt.subplot(111)
ax.bar(cnt_srs.index, cnt_srs.values, color='seagreen')
ax.xaxis_date()
plt.xticks(rotation='vertical')
plt.show()

All listings have been created in April to July 2016 period in our data.

**Hour**
------

In [ ]:
hourDF = train.groupby(['hour', 'interest_level'])['hour'].count().unstack('interest_level').fillna(0)
hourDF[['low','medium',"high"]].plot(kind='bar', stacked=True);

**Month**
-------

In [ ]:
monthDF = train.groupby(['month', 'interest_level'])['month'].count().unstack('interest_level').fillna(0)
monthDF[['low','medium',"high"]].plot(kind='bar', stacked=True);

**Photos**
--------

In [ ]:
train.num_photos.value_counts().plot(kind='bar',color='purple')

In [ ]:
train['num_photos'].ix[train['num_photos']>15] = 15
sns.stripplot(y="num_photos", x="interest_level",data=train,jitter=True,order=order);

Data is evenly distributed across interest levels

*Features*
----------

In [ ]:
train.num_features.value_counts().plot(kind='bar',color='orange')
plt.xlabel("Number of features")

In [ ]:
train['num_features'].ix[train['num_features'] > 16] = 16
sns.stripplot(x="num_features", y="interest_level",data=train,jitter=True,order=order);

Data is evenly distributed across all levels

**High profile Managers**
--------------------

In this I have tried to find top managers with more than 80 listings altogether with low,medium or high interest levels and more than 30 listings with high interest levels.

In [ ]:
managerDF = train.groupby(['manager_id', 'interest_level'])['manager_id'].count().unstack('interest_level').fillna(0)
managerDF["sum"]=managerDF.sum(axis=1)
x=managerDF[ (managerDF['sum'] > 80) & (managerDF['high'] > 30)]
managerDF.head()
x.sort_values("sum",inplace=True)
plt.title("High Profile Managers")
plt.xlabel("High Interest level")
plt.ylabel("Manager ID")
x = x[x.index != 'e6472c7237327dd3903b3d6f6a94515a']
x["sum"].plot(kind='barh',color="brown");

**High profile Buildings**
=====================

In this I have tried to find top buildings with more than 60 listings altogether with low,medium or high interest levels and more than 15 listings with high interest levels.

In [ ]:
buildingDF = train.groupby(['building_id', 'interest_level'])['building_id'].count().unstack('interest_level').fillna(0)
buildingDF["sum"]=buildingDF.sum(axis=1)
x=buildingDF[ (buildingDF['sum'] > 60) & (buildingDF['high'] > 15)]
buildingDF.head()
#removing the outlier
x = x[x.index != '0']
x.sort_values("sum",inplace=True)
plt.title("High Profile Buildings")
plt.xlabel("High Interest level")
plt.ylabel("Building ID")
x["sum"].plot(kind='barh',color="orange");

**BedRooms , Bathrooms, Features , Price and Interest level all together**
-----------------------------------------------------------------------

In [ ]:
hover_text = []
bubble_size = []
slope = 2.8
for index, row in train.iterrows():
    hover_text.append(('Interest: {interest}<br>'+
                      'Bedrooms: {bedrooms}<br>'+
                      'Price: {price}<br>'+
                      'Bathrooms: {bathrooms}<br>'+
                      'Year: {year}').format(interest=row['interest_level'],
                                            bedrooms=row['bedrooms'],
                                            price=row['price'],
                                            bathrooms=row['bathrooms'],
                                            year=row['created']))
    bubble_size.append(len(row['features'])*slope)

train['text'] = hover_text
train['bubble_size'] = bubble_size

More to come. Please upvote if you find it useful.